<a href="https://colab.research.google.com/github/mbagci06/lung_covid/blob/main/Unet_train_lung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# BASICS 
import os
import numpy as np
import torch


# MODEL and TRAIN
import torchvision
import torchvision.models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset
import random
from skimage import io
import torch.nn as nn
import torch.nn.functional as F
from matplotlib import pyplot as plt
from torch import optim
from PIL import Image

In [2]:
! git clone https://github.com/mbagci06/lung_covid
! pip install kaggle
! mkdir ~/.kaggle
! cp lung_covid/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d anasmohammedtahir/covidqu

fatal: destination path 'lung_covid' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
mkdir: cannot create directory ‘/root/.kaggle’: File exists
covidqu.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
# The last part to hide output bacause it is more then 30000 lines
!unzip "/content/covidqu.zip" -d "/content/archive/" &> /dev/null

In [4]:
# running  .ipynb scripts to call their functions
%run lung_covid/functions.ipynb
%run lung_covid/unet.ipynb

# !cp -avr lung_covid/model/ .
# from model.unet import UNet, PretrainedUNet

In [5]:
# The Calling the validaton dataset
val=Lung_Dataset_RGB(split = "Val",class_folder='lung',shuffle=True,sample_size=700)# The length of the validation dataset to calculate the average value of the loss and accuracy
# The size of the dataset for calculate average datas
val_size=val.__len__()
# The Train dataset 
tra=Lung_Dataset_RGB(split = "Train",class_folder='lung',shuffle=True,sample_size=4000)
tra_size=tra.__len__()
# # The transform functions to conver the PIL images to the tensors


# The device looks for GPU if cannot find goes with CPU
device=torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# The model is called by this function from the model.py file 
net = Unet_Res().to(device)
# The adam optimizer for optimization the values will be same for the other model training
optimizer = optim.Adam(net.parameters(), lr= 0.0001)
# The Loss function Cross entropy the value 
criterion = nn.CrossEntropyLoss()


Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth


  0%|          | 0.00/171M [00:00<?, ?B/s]

In [6]:
batch_size=30
# loading images and masks for mini batches
data_loader = torch.utils.data.DataLoader(
        tra, batch_size=batch_size,  num_workers=2, drop_last=True)

data_loader_val = torch.utils.data.DataLoader(
        val, batch_size=batch_size,  num_workers=2, drop_last=True)

In [7]:
# The stiation of the GPU
!nvidia-smi

Wed Jun  8 04:30:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    33W / 250W |   1185MiB / 16280MiB |     14%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
# The number of the epoch 
epoch_number=30
# list for the calculating validation Loss
val_loss_array=[]
# list for the calculating train Loss
train_loss_array=[]
# list for the calculating validation accuracy
acc_val_array=[]
# list for the calculating train accuracy
acc_train_array=[]
for t in range(epoch_number):
    # reset the loss and the accuracy values each epoch 
    val_loss=0
    train_loss=0
    acc_val=0
    acc_train=0
    for batch_idx, sample in enumerate(data_loader):
        # Train data preparing 
        imgs , true_masks = sample['image'],sample['mask']
        imgs = imgs.to(device=device)
        # The negative takes the masks and doubles the channels with inverse (0 to 1) shape  
        true_masks=Negative_mask(true_masks,imgs.size(dim=0))
        true_masks = true_masks.to(device=device)
        net.train()
        # zero the parameter gradients
        optimizer.zero_grad()
        

        with torch.set_grad_enabled(True):
            # Train data evaluation
            masks_pred = net(imgs)
            loss = criterion(masks_pred, true_masks)
            train_loss += loss.item()
            # Updating the params
            loss.backward()
            optimizer.step()
            # Acc calc train
            acc_train+=binary_acc( masks_pred, true_masks).item()
    for batch_idx, sample in enumerate(data_loader_val):
        # Validation data preparing  and calling 
        # calcel gradient feature 
        net.eval()

        # Call iamge and mask
        imgs , true_masks = sample['image'],sample['mask']

        imgs = imgs.to(device=device)
        true_masks=Negative_mask(true_masks,imgs.size(dim=0))
        true_masks = true_masks.to(device=device)

        # zero the parameter gradients
        optimizer.zero_grad()

        with torch.set_grad_enabled(False):
            # Validation evaluation 
            masks_pred = net(imgs)
            masks_pred = masks_pred.to(device=device)
            loss_val = criterion(masks_pred, true_masks)
            acc_val +=binary_acc( masks_pred, true_masks).item()
            val_loss += loss_val.item()

    batch_size
    print('Epoch :',t+1,'/',str(epoch_number))
    print('Validation Loss:v',(val_loss)/(val_size/batch_size))
    print('Validation Acc:',(acc_val)/(val_size/batch_size))
    print('Train Acc:',(acc_train)/(tra_size/batch_size))
    print('Train  Loss   :t',(train_loss)/(tra_size/batch_size))
    # Store epoch progress
    val_loss_array.append((val_loss)/(val_size/batch_size))
    train_loss_array.append((train_loss)/(tra_size/batch_size))
    acc_val_array.append((acc_val)/(val_size/batch_size))
    acc_train_array.append((acc_train)/(tra_size/batch_size))

Epoch : 1 / 30
Validation Loss:v 0.11903133222035
Validation Acc: 0.9677295786993845
Train Acc: 0.8381665286421776
Train  Loss   :t 0.2811109758540988
Epoch : 2 / 30
Validation Loss:v 0.052210336178541185
Validation Acc: 0.9728774326188224
Train Acc: 0.9818986201286315
Train  Loss   :t 0.08059492039494216
Epoch : 3 / 30
Validation Loss:v 0.03841131388076714
Validation Acc: 0.9734123647212982
Train Acc: 0.9851127339899539
Train  Loss   :t 0.04350704639218747
Epoch : 4 / 30
Validation Loss:v 0.03467996064573527
Validation Acc: 0.9726224797112601
Train Acc: 0.9864907206594943
Train  Loss   :t 0.031576944552361964
Epoch : 5 / 30
Validation Loss:v 0.030922467687300274
Validation Acc: 0.9734358608722687
Train Acc: 0.9873286239802836
Train  Loss   :t 0.02609216729644686
Epoch : 6 / 30
Validation Loss:v 0.028007876154567516
Validation Acc: 0.9741518855094911
Train Acc: 0.9878521567583083
Train  Loss   :t 0.023082547648809848
Epoch : 7 / 30
Validation Loss:v 0.026929332768278465
Validation Acc:

In [10]:
import copy
# Save Model to the drive folder 
best_model_wts = copy.deepcopy(net.state_dict())
torch.save({    
    'epoch':epoch_number,
    'model_state_dict':best_model_wts,
    'optimizer_state_dict': optimizer.state_dict()
     
},os.path.join(os.getcwd(),'UNet_model_lung_pre_{}_epoch.pt'.format(epoch_number)))

In [11]:
# Save loss and acc information to the txt 
textfile = open("UNet_model_lung_pre_{}_epo.txt".format(epoch_number), "w")
# The form of the saved values are important when we used them to plot
biglist=val_loss_array+   train_loss_array+    acc_val_array+    acc_train_array
textfile. write("running_loss_array+   train_loss_array+    acc_val_array+    acc_train_array"+ "\n")
for element in [biglist]:
    textfile. write(str(element) + "\n")
textfile. close()
